# 用Pythonic方式来思考

## 一. bytes、str与unicode的区别
Python3：有两种表示字符序列的类型：bytes和str：
- bytes：8个二进制位
- str：unicode字符  

Python2：有两种表示字符序列的类型：str和unicode：
- str：8个二进制位
- unicode：unicode字符

unicode到二进制：  
常见的编码方式是utf-8，encode方法  

二进制到unicode：  
decode方法

在编程的时候，编解码放在接口外面来做。程序核心使用unicode字符，且不要对字符编码做任何假设。

In [1]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decoce('utf-8')
    else:
        value = bytes_or_str
    return value

def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value

在Python3中，涉及到文件处理的操作（使用内置的open函数）会默认的以UTF-8进行编码。而在Python2中默认采用二进制形式来编码。这也是导致意外事故发生的根源，特别是对于那些更习惯于使用Python2的程序员而言。

比方说，将几个随机的二进制数据写入到一个文件中。在Python2中，下面的这段代码可以正常的工作，但是在Python3中却会报错并退出。

In [12]:
import os
with open('random.bin', 'w') as f:
    f.write(os.urandom(10))

TypeError: write() argument must be str, not bytes

导致这个异常发生的原因是在Python3中对于open函数又新增了一个名为encoding的参数。此参数默认为UTF-8。这样在文件句柄上进行read和write操作时，必须传入Unicode字符串的str实例，而不是包含了二进制数据的bytes实例。

## 二. 用生成器表达式来改写数据量较大的列表推导  


In [15]:
value = [len(x) for x in open('my_file.txt')]
value

[2, 3, 4, 5, 6, 7, 8, 9, 10, 3, 5, 7, 5, 5, 8, 11, 9, 3, 2, 2, 5, 5, 2]

- 使用圆括号构成生成器表达式

In [16]:
value = (len(x) for x in open('my_file.txt'))
value

<generator object <genexpr> at 0x00000278C2EBE360>

In [17]:
next(value)

2

In [18]:
next(value)

3

- 使用生成器表达式的另一个好处是可以互相组合

In [19]:
roots = ((x, x**10) for x in value)
roots

<generator object <genexpr> at 0x00000278C2EBE258>

In [20]:
next(roots)

(4, 1048576)

## 三. 尽量用enumerate代替range

In [22]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for i in range(len(flavor_list)):
    print('%d: %s' % (i+1, flavor_list[i]))

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [25]:
for i, flavor in enumerate(flavor_list):
    print('%d: %s' % (i+1, flavor))

1: vanilla
2: chocolate
3: pecan
4: strawberry


- 还可以直接指定enumerate函数开始计数时所用的值

In [26]:
for i, flavor in enumerate(flavor_list, 1):
    print('%d: %s' % (i, flavor))

1: vanilla
2: chocolate
3: pecan
4: strawberry


## 四. 用zip函数同时遍历两个迭代器

In [28]:
names = ['LiMing', 'LiLi', 'ZhangMei']
letters = [len(n) for n in names]
longest_name = None
max_letters = 0

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        max_letters = count
        longest_name = names[i]
    
        
print(longest_name, max_letters)

ZhangMei 8


- Python3中内置的zip函数，可以把两个或两个以上的迭代器封装为生成器，以便稍后求值，这种zip生成器，会从每个迭代器中获取该迭代器的下一个值，然后把这些值汇聚成元祖

In [29]:
longest_name = None
max_letters = 0
for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count
        
print(longest_name, max_letters)

ZhangMei 8


## 五. 不要在for和while循环后面写else块

## 六. 合理利用 tyr/except/else/finally 结构中的每个代码块

- finally

In [35]:
handle = open('my_file.txt')
try:
    data = handle.read()
finally:
    print('close handle')
    handle.close()

close handle


- else

In [36]:
import json
def load_json_key(data, key):
    try:
        result_dict = json.load_json_key(data)
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key]

- 混合使用

In [37]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')
    try:
        data = handle.read()
        op = json.loads(data)
        value = (op['numerator'], op['denominator'])
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        handle.close()

## 七. 尽量用异常来表示特殊情况，而不要返回None 

In [41]:
def devide(a, b):
    try:
        return a / b
    except:
        return None

def divide(a, b):
    try:
        return True, a/b
    except ZeroDivisionError:
        return False, None
    
sucess, result = divide(3,3)
result

1.0

In [43]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

try:
    result = divide(3, 0)
except ValueError:
    print('Invlid inputs')
else:
    print('Result is: %f' % result)

Invlid inputs
